<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time

import pandas_datareader as pdr
%tensorflow_version 2.x
import keras

from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
#not all needed

TensorFlow 2.x selected.


Using TensorFlow backend.


In [10]:
print(keras.__version__)

2.2.5


In [0]:
#Data import from git repo
url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
#df1 = pd.read_csv(url, error_bad_lines=False)
dfstocks = pd.read_csv(url, sep=',')

In [3]:
#fix column names
dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [5]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

229286

In [6]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
1761,01/02/2013,78.4328,139906732,79.1171,79.2857,77.3757,AAPL
4248,01/02/2013,35.12,13767660,34.92,35.4,34.1,ABBV
6025,01/02/2013,32.05,20266410,32.3,32.45,31.64,ABT
8512,01/02/2013,69.06,4039095,67.59,69.06,67.55,ACN
10999,01/02/2013,38.34,6483720,37.92,38.73,37.92,ADBE
...,...,...,...,...,...,...,...
10747,01/02/2014,59.29,2745895,59.06,59.53,58.94,ADBE
13234,01/02/2014,168.05,1268722,167.33,170.71,166.5937,AGN
15721,01/02/2014,50.71,9196092,50.81,51.3,50.47,AIG
18208,01/02/2014,53.55,1909106,54.09,54.45,53.5,ALL


In [72]:
#Test Split
dfstocks_split = dfstocks.sample(frac=0.9998,random_state=200)
second_split=dfstocks.drop(dfstocks_split.index)
# second_split = second_split.reindex(index=second_split.Date(second_split.index.min(), 
#                                           second_split.index.max(), 
#                                           freq='D')).fillna(method='ffill')
second_split

,Date,Close,Volume,Open,High,Low,Company
159523,01/04/2016,103.57,1929089,102.87,103.65,102.34,NEE
173214,01/05/2011,94.1728,3597597,93.3963,94.7261,92.5616,OXY
15212,01/11/2016,57.69,9615965,57.89,58,56.84,AIG
95258,01/15/2015,23.5037,31431330,23.7828,23.8925,23.4838,GE
219831,01/16/2018,133.97,5170230,137.73,137.73,133.87,UTX
38340,01/25/2013,718.82,1570859,682,719.37,681.05,BKNG
183867,02/05/2018,74.705,18709640,75.95,78.98,74.68,PYPL
211127,02/06/2013,57.38,5266009,57.44,57.54,57.1,UNH
214108,02/17/2011,48.91,6399682,48.77,48.98,48.165,UNP
166448,02/20/2018,249.08,17117670,244.75,251.87,244.6,NVDA


In [73]:
second_split.shape[0]
second_split.isnull().values.any()
second_split.isna().values.any()
second_split.reset_index(drop=True, inplace=True)
second_split

,Date,Close,Volume,Open,High,Low,Company
0,01/04/2016,103.57,1929089,102.87,103.65,102.34,NEE
1,01/05/2011,94.1728,3597597,93.3963,94.7261,92.5616,OXY
2,01/11/2016,57.69,9615965,57.89,58,56.84,AIG
3,01/15/2015,23.5037,31431330,23.7828,23.8925,23.4838,GE
4,01/16/2018,133.97,5170230,137.73,137.73,133.87,UTX
5,01/25/2013,718.82,1570859,682,719.37,681.05,BKNG
6,02/05/2018,74.705,18709640,75.95,78.98,74.68,PYPL
7,02/06/2013,57.38,5266009,57.44,57.54,57.1,UNH
8,02/17/2011,48.91,6399682,48.77,48.98,48.165,UNP
9,02/20/2018,249.08,17117670,244.75,251.87,244.6,NVDA


In [0]:
TRAIN_PERCENT = 0.7
#STOCK_INDEX = '^GSPC'
VERBOSE=True

In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def get_seq_train_test(time_series, scaling=True,train_size=0.9):
def get_seq_train_test(time_series,train_size=0.9):
    # scaler = None
    # if scaling:
    #     scaler = MinMaxScaler(feature_range=(0, 1))
    #     time_series = np.array(time_series).reshape(-1,1)
    #     scaled_stock_series = scaler.fit_transform(time_series)
    # else:
    #     scaled_stock_series = time_series
    scaled_stock_series = time_series
        
    #train_size = int(len(scaled_stock_series) * train_size)
    train_size = int(len(time_series) * train_size)

    #train = scaled_stock_series[0:train_size]
    train = time_series[0:train_size]
    #test = scaled_stock_series[train_size:len(scaled_stock_series)]
    test = time_series[train_size:len(time_series)]
    
    return train,test 


In [74]:
# split train and test datasets
train,test = get_seq_train_test(second_split.Close,
#                                  scaling=True,
                                   train_size=TRAIN_PERCENT)

#train
print(len(train))
print(len(train.index))
print(train.shape[0])
print(test.shape[0])

32
32
32
14


In [75]:
train = np.reshape(train,(1,train.shape[0],1))
test = np.reshape(test,(1,test.shape[0],1))

train_x = train[:,:-1,:]
train_y = train[:,1:,:]

test_x = test[:,:-1,:]
test_y = test[:,1:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

ValueError: ignored